# Google Calendar to Gridy

- 指定したカレンダーのカレンダーIDを取得
- APIを使って情報を抜き出す
- Gridyにインポートできるように整形
- seleniumとかでぶち込む？（APIあるかな）

In [1]:
import json

from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import sys

In [2]:
# 抜き出すCalendarIDはJsonファイルで保存
with open("../calendar_info.json","r") as f:
    calendar_info = json.load(f)

```Python
calendar_info = {任意の名前: カレンダーID, ...}
```

In [3]:
# この辺りはテンプレ

# これらのスコープを変更する場合は、token.pickleというファイルを削除してください。
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

creds = None
# token.pickleファイルには、ユーザーのアクセストークンと
# リフレッシュトークンが格納されており、認証フローが初めて完了したときに
# 自動的に作成されます。
if os.path.exists('../token.pickle'):
    # gitにupする関係上ログイン関係ファイルは1階層下に配置している
    with open('../token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# 利用可能な（有効な）認証情報がない場合は、ユーザーのログインを許可します。
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('../token.pickle', 'wb') as token:
        pickle.dump(creds, token)
        
# たぶん、カレンダーだよって主張している
service = build('calendar', 'v3', credentials=creds)

In [4]:
# Calendar APIの呼び出し
# 現在時刻
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

yotei = {}
# カレンダーIDすべての予定を抽出
for ID in calendar_info.values():
    events_result = service.events().list(calendarId=ID, timeMin=now,
                                        maxResults=100, singleEvents=True,
                                        orderBy='startTime').execute()
    for event in events_result.get('items', []):
        yotei[event["id"]] = {'summary':event['summary'],
                              "start":event["start"],
                              "end":event["end"]}

- 項目名	説明
- BEGIN:VCALENDAR*	先頭行に入力します。
- BEGIN:VEVENT	スケジュール情報の先頭行に入力します。
- 複数のスケジュールをインポートすることも可能です。その場合、BEGIN:VEVENTから- END:VEVENTを繰り返し入力してください。
- SUMMARY:*	スケジュールのタイトルです。(100文字まで)
- DTSTART:*	スケジュールの開始日時です。
- DTEND:*	スケジュールの終了日時です。
- LOCATION:	スケジュールの住所です。(100文字まで)
- DESCRIPTION:	スケジュールのメモです。
- 改行コードを使用する場合は、\(バックスラッシュ)nを使用してください。(10000文字まで)
- END:VEVENT	スケジュール情報の最終行に入力します。
- END:VCALENDAR*	最終行に入力します。

| インポートするスケジュール | 日時:2018年7月25日10:00から11:30<br />タイトル:会議<br />日時:2018年8月1日14:00から15:00<br />タイトル:外出<br />住所:東京都港区<br />メモ:1行目<br />2行目<br />3行目 |
| -------------------------- | ------------------------------------------------------------ |
| iCalファイル入力例         | BEGIN:VCALENDAR<br/>BEGIN:VEVENT<br/>SUMMARY:会議<br/>DTSTART:20180725T100000<br/>DTEND:20180725T113000<br/>END:VEVENT<br/>BEGIN:VEVENT<br/>SUMMARY:外出<br/>DTSTART:20180801T140000<br/>DTEND:20180801T150000<br/>DESCRIPTION:1行目\n2行目\n3行目<br/>LOCATION:東京都港区<br/>END:VEVENT<br/>END:VCALENDAR |



In [5]:
# インポートするファイルの中身
yotei_iCal_text = ""
# 最初に書くおまじない
ini = """BEGIN:VCALENDAR\n
"""
# 最後に書くおまじない
end = """
END:VCALENDAR
"""

# iCalファイルの中身を書き出し
yotei_iCal_text += ini

for event in yotei.values():
    # おまじない
    yotei_iCal_text += "BEGIN:VEVENT\n"
    # タイトル
    yotei_iCal_text += "SUMMARY:"+event["summary"]+"\n"
    # start
    # dateTimeの時の処理
    if list(event["start"].keys())[0] == "dateTime":
        date,time = event["start"]["dateTime"].split("T")
        year,month,day = date.split("-")
        time = time.split("+")[0]
        hour,minit,second = time.split(":")

        yotei_iCal_text += "DTSTART:"+year+month+day+"T"+hour+minit+second+"\n"
    # dateの時の処理
    if list(event["start"].keys())[0] == "date":
        year,month,day = event["start"]["date"].split("-")

        yotei_iCal_text += "DTSTART:"+year+month+day+"T000000\n"
    # end
    # dateTimeの時の処理
    if list(event["end"].keys())[0] == "dateTime":
        date,time = event["end"]["dateTime"].split("T")
        year,month,day = date.split("-")
        time = time.split("+")[0]
        hour,minit,second = time.split(":")

        yotei_iCal_text += "DTEND:"+year+month+day+"T"+hour+minit+second+"\n"
    # dateの時の処理
    if list(event["end"].keys())[0] == "date":
        year,month,day = event["end"]["date"].split("-")

        yotei_iCal_text += "DTEND:"+year+month+day+"T000000\n"
    # おまじない
    yotei_iCal_text += "END:VEVENT\n"
    
yotei_iCal_text += end

In [6]:
print(yotei_iCal_text)

BEGIN:VCALENDAR

BEGIN:VEVENT
SUMMARY:🦋Morfonica ZEPP TOUR 2021「Amabile」大阪公演
DTSTART:20211028T000000
DTEND:20211029T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🦋Morfonica ZEPP TOUR 2021「Amabile」東京公演
DTSTART:20211105T000000
DTEND:20211106T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:内定者懇談会
DTSTART:20211110T133000
DTEND:20211110T153000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🎧RAISE A SUILEN ZEPP TOUR 2021「BE LIGHT」追加公演 ＠Zepp Sapporo
DTSTART:20211112T000000
DTEND:20211113T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🎧RAISE A SUILEN ZEPP TOUR 2021「BE LIGHT」追加公演 ＠Zepp Fukuoka
DTSTART:20211126T000000
DTEND:20211127T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🎧RAISE A SUILEN ZEPP TOUR 2021「BE LIGHT」追加公演 ＠KT Zepp Yokohama
DTSTART:20211204T000000
DTEND:20211205T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🌹2021年Roselia単独ライブ
DTSTART:20211211T000000
DTEND:20211212T000000
END:VEVENT
BEGIN:VEVENT
SUMMARY:🌹2021年Roselia単独ライブ
DTSTART:20211212T000000
DTEND:20211213T000000
END:VEVENT

END:VCALENDAR



In [7]:
with open("../import.ics","w", encoding="utf8") as f:
    f.write(yotei_iCal_text)